In [ ]:
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd  /content/drive/Shared\ drives/256/ASKus

/content/drive/Shared drives/256/ASKus


In [6]:
#check num json files in train (should be 14,316)
import glob
len(glob.glob1("./data/train", "*.json"))

14316